In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random

In [2]:
all_a = []
limit = 30
for i in tqdm(range(0, 100000, limit)):
    if i == 0:
        i = ''
    else:
        i = f'/+{i}'
    url = f'https://forum.lowyat.net/PolitiKs{i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and a_.get('href').startswith('/topic')]
    a = list(set([a_ for a_ in a if len(a_.split('/')) == 3]))
    if not len(a):
        break
    all_a.extend(a)

 10%|███▉                                    | 332/3334 [04:45<43:05,  1.16it/s]


In [6]:
all_a = list(set(all_a))

In [7]:
len(all_a)

9958

In [8]:
with open('PolitiKs-topics.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    try:
        r_ = requests.get(url)
        soup = BeautifulSoup(r_.content, "lxml")
        divs = soup.find_all('div', {'class': 'post_text'})
        for div in divs:
            t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
            if len(t):
                texts.append(t)
    except Exception as e:
        pass
    return texts

In [12]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for topic in tqdm(sorted(all_a)):
    
    base_dir = topic[1:].replace('/', '-')
    os.makedirs(base_dir, exist_ok = True)
    os.makedirs('done-PolitiKs', exist_ok = True)
    
    base_dir_filename = os.path.join('done-PolitiKs', f'{base_dir}.json')
    
    if os.path.exists(base_dir_filename):
        continue
    
    need_break = False
    last_texts = []
    for i in range(0, 100000, per_batch):
        filename = os.path.join(base_dir, f'{i}.json')
        if os.path.exists(filename):
            continue
        aranged = np.arange(i, i + per_batch, limit)
        urls = []
        for a_ in aranged:
            if a_ == 0:
                a_ = ''
            else:
                a_ = f'/+{a_}'
            url = f'https://forum.lowyat.net/{topic}{a_}'
            urls.append(url)
        
        texts = []
        
        with ThreadPoolExecutor(max_workers=worker_size) as executor:
            futures = {executor.submit(get_text, item): item for item in urls}
            
            for future in as_completed(futures):
                r = future.result()
                texts.extend(r)
                if not len(r):
                    need_break = True
                    
                s = set(texts)
                if len(set(last_texts) & s) == len(s):
                    need_break = True
                    
                last_texts = texts[:]
        
        with open(filename, 'w') as fopen:
            json.dump(texts, fopen)
            
        time.sleep(random.uniform(0.5, 1.5))
        
        if need_break:
            break
    
    with open(base_dir_filename, 'w') as fopen:
        json.dump({'status': True}, fopen) 

  2%|▋                                    | 169/9958 [18:38<15:22:38,  5.66s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 17%|█████▉                            | 1742/9958 [3:16:33<13:10:21,  5.77s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 18%|██████                            | 1766/9958 [3:18:57<12:07:47,  5.33s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, byt

 22%|███████▎                          | 2157/9958 [3:59:21<16:16:59,  7.51s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 22%|███████▎                          | 2160/9958 [3:59:40<15:11:35,  7.01s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input err

 28%|█████████▍                        | 2747/9958 [5:02:13<19:04:18,  9.52s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 28%|█████████▋                         | 2759/9958 [5:03:15<7:58:25,  3.99s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
 28%|█████████▍                        | 2761/9958 [5:03:33<13:42:35,  6.86s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, byt

 35%|███████████▊                      | 3447/9958 [6:34:00<14:39:48,  8.11s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x9D 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x9D 0x3C 0x62 0x72
 35%|███████████▊                      | 3453/9958 [6:34:54<14:21:01,  7.94s/it]encoding error : input conversion failed due to input error, bytes 0x8F 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x8F 0x3C 0x62 0x72
encoding error : input conversion failed due to input error, bytes 0x8F 0x3C 0x62 0x72
 35%|███████████▊                      | 3466/9958 [6:36:53<12:43:23,  7.06s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, byt

 66%|██████████████████████▍           | 6589/9958 [12:31:52<8:46:29,  9.38s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
 68%|███████████████████████           | 6750/9958 [12:50:42<6:11:06,  6.94s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 68%|███████████████████████▏          | 6804/9958 [12:56

 83%|████████████████████████████▏     | 8244/9958 [15:26:39<3:27:59,  7.28s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 83%|████████████████████████████▎     | 8282/9958 [15:31:20<3:40:24,  7.89s/it]encoding error : input conversion failed due to input err

 90%|██████████████████████████████▌   | 8962/9958 [16:40:31<1:40:30,  6.05s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 90%|██████████████████████████████▌   | 8963/9958 [16:40:47<2:29:16,  9.00s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6B 0x65
 90%|██████████████████████████████▌   | 8965/9958 [16:40

 92%|███████████████████████████████▎  | 9175/9958 [17:03:33<1:31:53,  7.04s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
 92%|███████████████████████████████▎  | 9178/9958 [17:03:51<1:17:07,  5.93s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x74 0x6F
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x74 0x6F
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x74 0x6F
 92%|███████████████████████████████▍  | 9193/9958 [17:05:40<1:13:13,  5.74s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6E 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x6E 0x65
 92%|███████████████████████████████▍  | 9196/9958 [17:06:04<1:2

In [13]:
!mkdir PolitiKs

In [15]:
!mv topic-* PolitiKs

In [16]:
!du -hs PolitiKs

220M	PolitiKs


In [18]:
# !zip -r PolitiKs.zip PolitiKs

In [19]:
from glob import glob

texts = []

files = glob('PolitiKs/topic-*/*.json')
for f in tqdm(files):
    with open(f) as fopen:
        texts.extend(json.load(fopen))

100%|██████████████████████████████████| 33479/33479 [00:00<00:00, 59671.52it/s]


In [20]:
texts = list(set(texts))
len(texts)

354042

In [21]:
texts[-10:]

['QUOTE(Evolving Skies @ Nov 6 2022, 08:13 PM)Dr Kow is miles better\n\n\nI see u..also pity u. Cr8 so many dupes, still let ppl find out.',
 'i believe dap in.\npas out',
 'QUOTE(pakmulau @ Aug 20 2021, 07:10 PM)before this pun muhyidin got umno support\n\nnow ismail got bersatu support\n\napa yg beza nya?\n\nfor me no umno no bersatu still stand\n\nbefore this drama pun no bersatu no umno\n\nwhere i said i like muhyidin or ismail?\nAnyone know what our friend here mean by no umno no bersatu? Im still trying to brain, lol 😅\n\nMust be some really good shit he smokin.\nBro, jom lepak one day, you"re a fun guy. Bring some of that shit also 😉😉',
 'jubur takleh keluar taik salah DAP juga ke saiful',
 'QUOTE(achong09 @ Nov 9 2022, 09:30 AM)apa lanjiu kj cakap?? no logic??? uncle also went to hear his speech... uncle shake head... apa lanjiao dia cakap???? brainless!!!\nWhich part u shakehead? Bout his plan to open autism care centre?',
 'QUOTE(ju146 @ May 8 2018, 10:52 PM)A leader that kee

In [22]:
with open('PolitiKs.json', 'w') as fopen:
    json.dump(texts, fopen)